Input
existing_table = [
{"id": 1, "value": 10, "updated_at": "2024-01-01T10:00:00"},
{"id": 2, "value": 20, "updated_at": "2024-01-02T10:00:00"},
]
incoming_batch = [
{"id": 2, "value": 25, "updated_at": "2024-01-03T09:00:00"},
{"id": 3, "value": 30, "updated_at": "2024-01-03T09:05:00"},
]
Description
Implement a Python function that merges the incoming batch into the existing dataset.
Requirements:
● Use id as the primary key
● Keep only the most recent record per id using updated_at
● The function must be idempotent
Return the updated dataset as a list of dictionaries

In [1]:
existing_table = [
{"id": 1, "value": 10, "updated_at": "2024-01-01T10:00:00"},
{"id": 2, "value": 20, "updated_at": "2024-01-02T10:00:00"},
]

incoming_batch = [
{"id": 2, "value": 25, "updated_at": "2024-01-03T09:00:00"},
{"id": 3, "value": 30, "updated_at": "2024-01-03T09:05:00"},
]

In [2]:
import pandas as pd
df1 = pd.DataFrame(data=existing_table)
df2 = pd.DataFrame(data =incoming_batch)

df2.head()

,id,value,updated_at
0,2,25,2024-01-03T09:00:00
1,3,30,2024-01-03T09:05:00


In [3]:
# date = max(df1['updated_at'].groupby(df1['id']))
# date

# df.groupby("Animal", group_keys=True)[["Max Speed"]].apply(lambda x: x
df2.groupby("id" )['updated_at'].min()

frames = [df1, df2]

result = pd.concat(frames)
# result
max_date_df1 = df1['updated_at'].max()
max_date_df1

'2024-01-02T10:00:00'

In [11]:
def batch_ingestion(existing_table,incoming_batch):
    df1 = pd.DataFrame(data=existing_table)
    df2 = pd.DataFrame(data =incoming_batch)

    # max_date_df1 = df1.groupby("id" )['updated_at'].max()
    max_date_df1 = df1['updated_at'].max()


    # min_date_df2 = df1.groupby("id" )['updated_at'].min()

    max_date_df2 = df2['updated_at'].max()

    if max_date_df2  > max_date_df1:

        frames = [df1, df2]

    result = pd.concat(frames)
    return result.to_dict(orient="records")


In [12]:
batch_ingestion(existing_table,incoming_batch)

[{'id': 1, 'value': 10, 'updated_at': '2024-01-01T10:00:00'},
 {'id': 2, 'value': 20, 'updated_at': '2024-01-02T10:00:00'},
 {'id': 2, 'value': 25, 'updated_at': '2024-01-03T09:00:00'},
 {'id': 3, 'value': 30, 'updated_at': '2024-01-03T09:05:00'}]

In [6]:
import pandas as pd

def batch_ingestion(existing_table, incoming_batch):
    """
    Merge incoming_batch into existing_table using:
      - id as primary key
      - keep only latest record per id by updated_at
      - idempotent (re-running with same batch doesn't change output)
    Returns: list[dict]
    """
    # Build DataFrames (handle empty inputs safely)
    df1 = pd.DataFrame(existing_table) if existing_table else pd.DataFrame(columns=["id","value","updated_at"])
    df2 = pd.DataFrame(incoming_batch) if incoming_batch else pd.DataFrame(columns=["id","value","updated_at"])

    # Ensure datetime for correct comparisons (supports ISO timestamps)
    df1["updated_at"] = pd.to_datetime(df1["updated_at"], errors="coerce", utc=False)
    df2["updated_at"] = pd.to_datetime(df2["updated_at"], errors="coerce", utc=False)

    # Combine
    combined = pd.concat([df1, df2], ignore_index=True)

    # If there are multiple rows per id, keep the most recent updated_at
    # Sort so latest per id comes first, then drop duplicates
    combined = combined.sort_values(["id", "updated_at"], ascending=[True, False])
    combined = combined.drop_duplicates(subset=["id"], keep="first")

    # Optional: sort output by id for stable deterministic output
    combined = combined.sort_values("id").reset_index(drop=True)

    # Convert updated_at back to ISO string (without timezone) for output
    combined["updated_at"] = combined["updated_at"].dt.strftime("%Y-%m-%dT%H:%M:%S")

    return combined.to_dict(orient="records")


In [13]:
import pandas as pd

def batch_ingestion(existing_table, incoming_batch):
    """
    Merge incoming_batch into existing_table using:
      - id as primary key
      - keep only latest record per id by updated_at
      - idempotent (re-running with same batch doesn't change output)
    Returns: list[dict]
    """
    # Build DataFrames (handle empty inputs safely)
    df1 = pd.DataFrame(existing_table) 
    df2 = pd.DataFrame(incoming_batch)

    # Ensure datetime for correct comparisons (supports ISO timestamps)
    df1["updated_at"] = pd.to_datetime(df1["updated_at"], errors="coerce", utc=False)
    df2["updated_at"] = pd.to_datetime(df2["updated_at"], errors="coerce", utc=False)

    # Combine
    combined = pd.concat([df1, df2], ignore_index=True)

    # If there are multiple rows per id, keep the most recent updated_at
    # Sort so latest per id comes first, then drop duplicates
    combined = combined.sort_values(["id", "updated_at"], ascending=[True, False])
    combined = combined.drop_duplicates(subset=["id"], keep="first")

    # Optional: sort output by id for stable deterministic output
    combined = combined.sort_values("id").reset_index(drop=True)

    # Convert updated_at back to ISO string (without timezone) for output
    combined["updated_at"] = combined["updated_at"].dt.strftime("%Y-%m-%dT%H:%M:%S")

    return combined.to_dict(orient="records")

In [14]:
batch_ingestion(existing_table, incoming_batch)

[{'id': 1, 'value': 10, 'updated_at': '2024-01-01T10:00:00'},
 {'id': 2, 'value': 25, 'updated_at': '2024-01-03T09:00:00'},
 {'id': 3, 'value': 30, 'updated_at': '2024-01-03T09:05:00'}]